# 7 Working with Keras: A deep dive

In [1]:
import sys

if "google.colab" in sys.modules:
    print("Running in Colab")
    !pip install tf-nightly

Running in Colab


In [2]:
import time

import tensorflow as tf

from IPython.display import clear_output

## 7.3 Using built-in training and evaluation loops

**The standard workflow: `compile()`, `fit()`, `evaluate()`, `predict()`**

In [3]:
def get_mnist_model():
    inputs = tf.keras.Input(shape=(28 * 28,))
    features = tf.keras.layers.Dense(512, activation="relu")(inputs)
    features = tf.keras.layers.Dropout(0.5)(features)
    outputs = tf.keras.layers.Dense(10, activation="softmax")(features)
    model = tf.keras.Model(inputs, outputs)
    return model

(images, labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

In [4]:
model = get_mnist_model()
model.compile(
    optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    train_images, train_labels, epochs=3, validation_data=(val_images, val_labels)
)
test_metrics = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.8639 - loss: 0.4432 - val_accuracy: 0.9597 - val_loss: 0.1394
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - accuracy: 0.9515 - loss: 0.1637 - val_accuracy: 0.9689 - val_loss: 0.1107
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9635 - loss: 0.1241 - val_accuracy: 0.9719 - val_loss: 0.1026
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9690 - loss: 0.1151
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


### 7.3.1 Writing your own metrics

**Implementing a custom metric by subclassing the `Metric` class**

In [5]:
class RootMeanSquaredError(tf.keras.metrics.Metric):
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32"
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)

    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

    def reset_state(self):
        self.mse_sum.assign(0.0)
        self.total_samples.assign(0)

In [6]:
model = get_mnist_model()
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy", RootMeanSquaredError()],
)
model.fit(
    train_images, train_labels, epochs=3, validation_data=(val_images, val_labels)
)
test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8640 - loss: 0.4518 - rmse: 0.4399 - val_accuracy: 0.9596 - val_loss: 0.1414 - val_rmse: 0.2485
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9529 - loss: 0.1610 - rmse: 0.2683 - val_accuracy: 0.9675 - val_loss: 0.1193 - val_rmse: 0.2226
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9645 - loss: 0.1294 - rmse: 0.2353 - val_accuracy: 0.9719 - val_loss: 0.1012 - val_rmse: 0.2058
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9693 - loss: 0.1161 - rmse: 0.2177


---

## 7.4 Writing your own training and evaluation loops

### 7.4.1 Training versus inference

### 7.4.2 Low-level usage of metrics

In [7]:
metric = tf.keras.metrics.SparseCategoricalAccuracy()
targets = [0, 1, 2]
predictions = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
metric.update_state(targets, predictions)
current_result = metric.result()
print(f"result: {current_result:.2f}")

result: 1.00


In [8]:
values = [0, 1, 2, 3, 4]
mean_tracker = tf.keras.metrics.Mean()
for value in values:
    mean_tracker.update_state(value)
print(f"Mean of values: {mean_tracker.result():.2f}")

Mean of values: 2.00


### 7.4.3 A complete training and evaluation loop

**Writing a step-by-step training loop: the training step function**

In [9]:
model = get_mnist_model()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.RMSprop()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
loss_tracking_metric = tf.keras.metrics.Mean()

def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metric.name] = metric.result()

    loss_tracking_metric.update_state(loss)
    logs["loss"] = loss_tracking_metric.result()
    return logs

**Writing a step-by-step training loop: resetting the metrics**

In [10]:
def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_tracking_metric.reset_state()

In [11]:
!python --version

Python 3.10.12


**Writing a step-by-step training loop: the loop itself**

In [12]:
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
training_dataset = training_dataset.batch(32).enumerate()
epochs = 3                                  # ↑ adds an index like Python enumerate
for epoch in range(epochs):
    reset_metrics()
    #   ↓ an index allows us to keep count
    for i, (inputs_batch, targets_batch) in training_dataset:
        clear_output(wait=True)
        print(f"{i+1}/{len(train_images)//32}")
        logs = train_step(inputs_batch, targets_batch)
    print(f"Results at the end of epoch {epoch}")
    for key, value in logs.items():
        print(f"...{key}: {value:.4f}")

1563/1562
Results at the end of epoch 2
...sparse_categorical_accuracy: 0.9641
...loss: 0.1304


**Writing a step-by-step evaluation loop**

In [13]:
def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()

    loss_tracking_metric.update_state(loss)
    logs["val_loss"] = loss_tracking_metric.result()
    return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()
times = []
for inputs_batch, targets_batch in val_dataset:
    t = time.time()
    logs = test_step(inputs_batch, targets_batch)
    times.append(time.time() - t)
print("Mean test_step time:", sum(times)/len(times))
print("Evaluation results:")
for key, value in logs.items():
    print(f"...{key}: {value:.4f}")

Mean test_step time: 0.007750961346367297
Evaluation results:
...val_sparse_categorical_accuracy: 0.9675
...val_loss: 0.1194


In [14]:
%%timeit
for inputs_batch, targets_batch in val_dataset:
    logs = test_step(inputs_batch, targets_batch)

2.91 s ± 895 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 7.4.4 Make it fast with tf.function

**Adding a `tf.function` decorator to our evaluation-step function**

In [15]:
@tf.function
def test_step_compiled(inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()

    loss_tracking_metric.update_state(loss)
    logs["val_loss"] = loss_tracking_metric.result()
    return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()
times = []
for inputs_batch, targets_batch in val_dataset:
    t = time.time()
    logs = test_step_compiled(inputs_batch, targets_batch)
    times.append(time.time() - t)
print("Mean test_step time:", sum(times)/len(times)) # an order of magnitude smaller
print("Evaluation results:")
for key, value in logs.items():
    print(f"...{key}: {value:.4f}")

Mean test_step time: 0.001947626138266664
Evaluation results:
...val_sparse_categorical_accuracy: 0.9675
...val_loss: 0.1194


In [16]:
%%timeit
for inputs_batch, targets_batch in val_dataset:
    logs = test_step_compiled(inputs_batch, targets_batch)

642 ms ± 257 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 7.4.5 Leveraging fit() with a custom training loop

**Implementing a custom training step to use with `fit()`**

In [17]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
loss_tracker = tf.keras.metrics.Mean(name="loss")

class CustomModel(tf.keras.Model):
    def train_step(self, data): # ← `train_step` is what we need to rewrit
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = loss_fn(targets, predictions)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))

        loss_tracker.update_state(loss)
        return {"loss": loss_tracker.result()}

    @property
    def metrics(self):
        return [loss_tracker]

In [18]:
inputs = tf.keras.Input(shape=(28 * 28,))
features = tf.keras.layers.Dense(512, activation="relu")(inputs)
features = tf.keras.layers.Dropout(0.5)(features)
outputs = tf.keras.layers.Dense(10, activation="softmax")(features)
model = CustomModel(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.RMSprop())
model.fit(train_images, train_labels, epochs=3)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.4492
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 0.1661
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.1296


Updated from DLWP, see [reference here](https://keras.io/guides/custom_train_step_in_tensorflow/#a-first-simple-example).

In [19]:
class CustomModel(tf.keras.Model):
    def train_step(self, data):
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            #           ↓ compute loss
            loss = self.compute_loss(y=targets, y_pred=predictions)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        #    ↓ compiled metrics for updating
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(targets, predictions)
        return {m.name: m.result() for m in self.metrics} # ← metrics for reporting

In [20]:
inputs = tf.keras.Input(shape=(28 * 28,))
features = tf.keras.layers.Dense(512, activation="relu")(inputs)
features = tf.keras.layers.Dropout(0.5)(features)
outputs = tf.keras.layers.Dense(10, activation="softmax")(features)
model = CustomModel(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.RMSprop(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
model.fit(train_images, train_labels, epochs=3)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - sparse_categorical_accuracy: 0.8698 - loss: 0.4427
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - sparse_categorical_accuracy: 0.9542 - loss: 0.1653
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - sparse_categorical_accuracy: 0.9624 - loss: 0.1290
